In [1]:
import json
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Polygon
from scipy.interpolate import interp2d

## Read data prior

In [11]:
unit_operator = 10**(-6) # unit Pa --> MPa
#unit_operator = 1
fim_converter = 1

# STD in MPa
#measurement_std = 0.00008
measurement_std = 0.0003

# MPa**2
measurement_var = measurement_std**2

print("Var:", measurement_var)

Var: 8.999999999999999e-08


In [12]:
# read prior
def read_fim(str_name, dim):
    f_prior = open(str_name)

    data_prior = json.load(f_prior)
    
    prior_info = data_prior[0]['Total']
    
    for i in range(dim):
        for j in range(dim):
            # unit transfer
            prior_info[i][j] *= unit_operator*unit_operator  
            # Var
            prior_info[i][j] *= 1/measurement_var
            
    print(prior_info)
    print("Determinant:", np.linalg.det(prior_info))
    print("Trace:", np.trace(prior_info))
    eig_vals = list(np.linalg.eigvals(prior_info))
    print(eig_vals)
    print("Condition number:", max(eig_vals)/min(eig_vals))
    
    print(np.linalg.eig(prior_info))
    
    
    return prior_info
    
    



In [19]:
num_para = 8  # FIM dimension

#file_name = 'emimtf2n_FIM_info/SRK_1Param_Opt2.json'
#file_name = 'emimtf2n_FIM_info/SRK_noTdep.json'
#file_name = '125_emimtf2n_FIM_info/PR-3A.json'
file_name = '32_bmimpf6_FIM_info/SRK-8-unscaleParam.json'


prior = read_fim(file_name, num_para)

[[99044852126.7339, -12522971179.441887, 200225478188.84824, 66758091044.93622, 326726119206.3942, 166995740540.64252, 18041941892218.406, -17264936573420.227], [-12522971179.441887, 2241493893.419018, -33787615583.502472, -16075013961.127308, -60159177011.172905, -38897037428.543724, -2750628808434.33, 2590577655210.126], [200225478188.84824, -33787615583.502472, 528012047658.1158, 247121795747.00192, 934913317236.452, 597642598938.6847, 43884296065095.42, -41411819165173.43], [66758091044.93622, -16075013961.127308, 247121795747.00192, 147224063031.83536, 469939156420.11646, 349437341561.3444, 18975625799702.4, -17631101438888.312], [326726119206.3942, -60159177011.172905, 934913317236.452, 469939156420.11646, 1689004027553.6682, 1130012575616.3843, 76075557657452.78, -71503543416940.52], [166995740540.64252, -38897037428.543724, 597642598938.6847, 349437341561.3444, 1130012575616.3843, 830731381976.0496, 46227169119934.83, -43013244533519.875], [18041941892218.406, -2750628808434.33

In [20]:

# covariance matrice

cov = np.linalg.pinv(prior)
print(cov)

[[ 1.70259018e-05 -1.20199324e-05 -3.22615099e-05  1.96164169e-05
   1.72719359e-05 -5.45357170e-06 -1.82745370e-06 -1.82749085e-06]
 [-1.20199595e-05  1.27833606e-05  2.05131991e-05 -2.39872389e-05
  -8.26029414e-06  1.12541801e-05 -4.31259270e-07 -4.31214231e-07]
 [-3.22614955e-05  2.05131375e-05  6.23585347e-05 -3.18279750e-05
  -3.48384559e-05  6.42981875e-06  4.37730730e-06  4.37736997e-06]
 [ 1.96164811e-05 -2.39872530e-05 -3.18281133e-05  4.66024280e-05
   1.06056777e-05 -2.38116414e-05  1.96679325e-06  1.96670701e-06]
 [ 1.72719109e-05 -8.26024894e-06 -3.48384231e-05  1.06055837e-05
   2.11670177e-05  1.25917872e-06 -3.44025355e-06 -3.44027643e-06]
 [-5.45361862e-06  1.12542045e-05  6.42991228e-06 -2.38116749e-05
   1.25912311e-06  1.45500446e-05 -2.39014769e-06 -2.39010438e-06]
 [-1.82744277e-06 -4.31269900e-07  4.37728813e-06  1.96681273e-06
  -3.44024515e-06 -2.39015622e-06  8.89267995e-07  8.89265062e-07]
 [-1.82747992e-06 -4.31224861e-07  4.37735081e-06  1.96672649e-06
  -

In [21]:
# correlation 

R = np.corrcoef(cov)

print(R)

[[ 1.         -0.93335409 -0.99646424  0.88103307  0.96161842 -0.64570279
  -0.56279222 -0.5628081 ]
 [-0.93335409  1.          0.89989529 -0.99212202 -0.79903635  0.87676476
   0.22857126  0.22858996]
 [-0.99646424  0.89989529  1.         -0.83817313 -0.98127206  0.57926499
   0.63025118  0.63026609]
 [ 0.88103307 -0.99212202 -0.83817313  1.          0.71741638 -0.93010478
  -0.10481252 -0.10483163]
 [ 0.96161842 -0.79903635 -0.98127206  0.71741638  1.         -0.41139889
  -0.76800172 -0.76801402]
 [-0.64570279  0.87676476  0.57926499 -0.93010478 -0.41139889  1.
  -0.26778451 -0.267766  ]
 [-0.56279222  0.22857126  0.63025118 -0.10481252 -0.76800172 -0.26778451
   1.          1.        ]
 [-0.5628081   0.22858996  0.63026609 -0.10483163 -0.76801402 -0.267766
   1.          1.        ]]


In [ ]:
eig_test = np.linalg.eig(prior)[1]

In [7]:
df = pd.DataFrame(R)
writer = pd.ExcelWriter('test.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='welcome', index=False)
writer.save()


##  Read data 

In [ ]:
# read prior

f = open('emimtf2n_FIM_info/PR_quad_543900_scaledBar_pa_scaleopt.json')
f2 = open('emimtf2n_FIM_info/PR_quad_543900_scaledBar_pa_scaleopt_pressure.json')
#f3 = open('emimtf2n_FIM_info/PR_quad_543900_scaledBar_under10.json')
#f4 = open('emimtf2n_FIM_info/PR_quad_543900_scaledBar_under10_pressure.json')

data = json.load(f)

data_pressure = json.load(f2)

#data_under10 = json.load(f3)
#data_pressure_under10 = json.load(f4)

In [ ]:
T_range_overall = [273, 283, 293, 303, 313, 323, 333, 343, 353, 363, 373, 383, 393, 400]
x_range_overall = [0.01, 0.05, 0.1,0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


In [ ]:
unit_operator = 10**(-6)
#unit_operator = 1
fim_converter = 1

# STD in MPa
measurement_std = 0.00008

# MPa**2
measurement_var = measurement_std**2

print("The unit operator converts units from Pa to Mpa:", unit_operator)
print(fim_converter)

In [ ]:
# read prior
def remove_fim(idx_to_remove):
    f_prior = open('emimtf2n_FIM_info/PR_quadTdep_scaleopt.json')

    data_prior = json.load(f_prior)

    #prior_info = data[0]['Total']
    prior_info = [[0]*6 for i in range(6)]
    
    # remove idx
    for idx in idx_to_remove:
        for a in range(6):
            for b in range(6):
                prior_info[a][b] += data_prior[0][str(idx)][a][b]

    for i in range(6):
        for j in range(6):
            prior_info[i][j] *= unit_operator*unit_operator      
            prior_info[i][j] *= 1/measurement_var
            
    return prior_info



In [ ]:
new_prior = [2, 3, 5, 12, 16, 19, 23, 26]
#new_prior = [32]
num_to_remove = []
for i in range(27):
    if i not in new_prior:
        num_to_remove.append(i)

fim_to_be_removed = remove_fim(num_to_remove)
print(fim_to_be_removed)

In [ ]:
def read_heatmap(T_range, x_range, data_set, pressure_data_set, remove=None):
    A_heatmap = np.zeros((len(T_range), len(x_range)))
    D_heatmap = np.zeros((len(T_range), len(x_range)))
    E_heatmap = np.zeros((len(T_range), len(x_range)))
    ME_heatmap = np.zeros((len(T_range), len(x_range)))

    pressure_heatmap = np.zeros((len(T_range), len(x_range)))

    for i in range(len(T_range)):
        for j in range(len(x_range)):
            count = i*len(x_range) + j 

            #fim = np.asarray(data_set[0][str(count)])
            # the recorded FIM (Q.T@Q) is based on MPa for measurements 
            # computed with 1 for meausrement STD 
            fim = np.asarray(data_set[0][str(count)])*fim_converter*fim_converter
            
            # factor in measurement STD; post processing step 
            fim *= 1/measurement_var
            
            print("before:", fim)
            
            # for the removing prior fn. Default is closed. 
            if remove:
                for a in range(6):
                    for b in range(6):
                        fim[a][b] -= remove[a][b]
                    
            fim = np.asarray(fim)
            print("after:", fim)
            
            
            A_heatmap[i,j] = np.trace(fim)
            D_heatmap[i,j] = np.linalg.det(fim)
            E_heatmap[i,j] = min(np.linalg.eigvals(fim))
            ME_heatmap[i,j] = np.linalg.cond(fim)

            # convert unit
            pressure_heatmap[i,j] = pressure_data_set[0][str(count)]*unit_operator
            
    return A_heatmap, D_heatmap, E_heatmap, ME_heatmap, pressure_heatmap

In [ ]:
A_heatmap_overall, D_heatmap_overall, E_heatmap_overall, ME_heatmap_overall, pressure_heatmap_overall = read_heatmap(T_range_overall, x_range_overall, data, data_pressure, remove=fim_to_be_removed)


In [ ]:
#print(pressure_heatmap)

A_practical  = A_heatmap_overall.copy()
D_practical  = D_heatmap_overall.copy()
E_practical  = E_heatmap_overall.copy()
ME_practical  = ME_heatmap_overall.copy()
pressure_practical = pressure_heatmap_overall.copy()

for i in range(len(pressure_heatmap_overall)):
    for j in range(len(pressure_heatmap_overall[0])):
        if pressure_practical[i,j] > 2.5:
            pressure_practical[i,j] = float("NaN")
            
            A_practical[i,j] = float('NaN')
            E_practical[i,j] = float('NaN')
            D_practical[i,j] = float('NaN')
            ME_practical[i,j] = float('NaN')
print(pressure_practical)

In [ ]:
data_file = './emimtf2n/R32/Final_Results/MBDoE/r32_emimtf2n_subset.csv'
data_exp = pd.read_csv(data_file)
print(len(data_exp))

In [ ]:
#print(D_practical)
import math
from scipy.interpolate import interp1d
#print(np.shape(D_practical))

T_range_lab = T_range_overall   
x_range_lab = [1, 5, 10,20,30,40,50,60,70,80,90]

edges = []
for i in range(14):
    for j in range(11):
        if math.isnan(D_practical[i,j]):
            #edges.append((i,j))
            edges.append((i, j-1))
            break;
            
#print(edges)

edges_map = []

for edge in edges:
    edges_map.append((T_range_lab[edge[0]], x_range_lab[edge[1]]))
    
#print(edges_map)

print(T_range_lab[edges[0][0]-1])

a, b = T_range_lab[edges[0][0]-1], edges_map[-1][1]

a_real = T_range_lab[edges[0][0]]

x =[edges_map[i][0] for i in range(len(edges_map))]
y =[edges_map[i][1] for i in range(len(edges_map))]
f = interp1d(x, y, kind='quadratic')

xnew = np.arange(a_real, 400)
ynew = f(xnew)   # use interpolation function returned by `interp1d`
#print(ynew)

#print(x, y)
#print(f)

In [ ]:
x_ax = T_range_lab
y_ax = x_range_lab 
z_ax = pressure_heatmap_overall

xnew = np.arange(285, 400, 0.1)
ynew = np.arange(18, 90, 0.1) 

f = interp2d(x_ax, y_ax, z_ax.T)

f_new = f(xnew, ynew)
print(np.shape(f_new))

f_newT =  f_new.T

x_15 = []
y_15 = []

for i in range(len(xnew)):
    for j in range(len(ynew)):
        if i>=1 and f_newT[i,j] >= 1.500000 and f_newT[i-1,j]<=1.500000:
            x_15.append(xnew[i])
            y_15.append(ynew[j])

In [ ]:
def contour(T_range_label, x_range_label, heatmap_data, bar_label, plot_prior=True):
    # draw figure, using (6,6) because the plot is small otherwise
    #fig, ax = plt.figure(figsize=(6,6))
    fig, ax = plt.subplots()
    
    # plot heatmap
    #   cmap defines the overall color within the heatmap 
    #   levels: determines the number and positions of the contour lines / regions.
    cs = plt.contourf(T_range_label, x_range_label, heatmap_data.T,cmap=cm.coolwarm)

    # plot color bar
    cbar = plt.colorbar(cs)

    # plot title in color bar
    cbar.ax.set_ylabel(bar_label, fontsize=16, fontweight='bold')

    # set font size in color bar
    cbar.ax.tick_params(labelsize=16)
    
    def func(x):
        return x*0.1

    #ix = np.linspace(a, b)
    #iy = func(ix)
    a_new = np.linspace(a, 400,1)
    upper_90 = [90 for i in range(len(a_new))]
    
    b_new = np.linspace(b, 90, 1)
    right_400 = [400 for j in range(len(b_new))]
    
    #verts = [(a, 90)] + list(zip(ix, iy)) + [(400, b)]
    #verts = list(zip(a_new, upper_90)) + list(zip(xnew, ynew)) + list(zip(right_400, b_new)) + [(400,90)]
    verts = list(zip(a_new, upper_90)) + list(zip(x_15, y_15)) + list(zip(right_400, b_new)) + [(400,90)]
    poly = Polygon(verts, facecolor='white', edgecolor='darkblue', closed=True)
    #polygon1 = Polygon([[0.5,0.2], [0.4,0.3], [0.6, 0.9],], closed=True, alpha=1)
    
    ax.add_patch(poly)

    # plot equipotential line
    #  [::10] means sampling 1 in every 10 samples
    #  colors define the color want to use, 'k' for black
    #  alpha is blending value, between 0 (transparent) and 1 (opaque).
    #  linestyle defines the linestyle.
    #  linewidth defines the width of line
    #cs2 = plt.contour(cs, levels=cs.levels[::10], colors='k', alpha=0.7, linestyles='dashed', linewidths=3)

    # plot the heatmap label
    #  %2.2f means keep to 2 digit
    #  fontsize defines the size of the text in figure
    #plt.clabel(cs2, fmt='%2.2f', colors='k', fontsize=16)

    # define tick size
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.tick_params(direction="in",top=True, right=True)

    # set squared figure
    if plot_prior:
        for i in new_prior:  
        #for i in range(len(data_exp)):
            if i==16:
                plt.scatter(data_exp.iloc[i]['temperature'], data_exp.iloc[i]['x_R32']*100, color="black",marker="*", s=150)
                
            elif i==3:
                plt.scatter(data_exp.iloc[i]['temperature'], data_exp.iloc[i]['x_R32']*100, color="black",marker="^", s=125)
                
            elif i in new_prior:
                plt.scatter(data_exp.iloc[i]['temperature'], data_exp.iloc[i]['x_R32']*100, color="black", s=80)
        
            #else:
            #plt.scatter(data_exp.iloc[i]['temperature'], data_exp.iloc[i]['x_R32']*100, color="darkgrey", s=20)
        
            
        
    # plot titile and x,y label
    plt.xlabel(r'Temperature [K]', fontsize=16, fontweight='bold')
    plt.ylabel(r'HFC-32 in IL [mol%]', fontsize=16, fontweight='bold')
    plt.show()

    # save figure (important for publications)
    # plt.savefig('heatmap.png',bbox_inches='tight')
    
    
T_range_lab = T_range_overall   
x_range_lab = [1, 5, 10,20,30,40,50,60,70,80,90]

#contour(T_range_lab, x_range_lab, np.log10(A_heatmap[:,:6]), 'log10(A-optimality)')
contour(T_range_lab, x_range_lab, np.log10(A_practical), 'log10(A-optimality)', plot_prior=True)
contour(T_range_lab, x_range_lab, np.log10(D_practical), 'log10(D-optimality)', plot_prior=True)
contour(T_range_lab, x_range_lab, np.log10(E_practical), 'log10(E-optimality)', plot_prior=True)



In [ ]:
contour(T_range_lab, x_range_lab, pressure_practical, 'Pressure [Mpa]', plot_prior=False)


## Draw prior pressure

In [ ]:
f5 = open('emimtf2n_FIM_info/PR_quadTdep_pressure.json')

data_prior = json.load(f5)

print(data_prior)
print(data_exp)

In [ ]:
def temperature_splitter(temp):

    temp_idx= data_exp.index[data_exp['temperature'] == temp].tolist()
    print(temp_idx)

    x = data_exp.loc[data_exp["temperature"]==temp]["x_R32"]

    y = [1E-6*data_prior[0][str(i)] for i in temp_idx]
    
    return x, y


In [ ]:
tmp283_x, tmp283_y = temperature_splitter(283.15)
tmp298_x, tmp298_y = temperature_splitter(298.15)
tmp323_x, tmp323_y = temperature_splitter(323.15)
tmp348_x, tmp348_y = temperature_splitter(348.05)

In [ ]:
plt.scatter(tmp283_x, tmp283_y, label="T=283 K")
plt.scatter(tmp298_x, tmp298_y, marker="*", label="T=298 K")
plt.scatter(tmp323_x, tmp323_y, marker="^", label="T=323 K")
plt.scatter(tmp348_x, tmp348_y, marker="s", label="T=348 K")

# define tick size
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tick_params(direction="in",top=True, right=True)

# plot titile and x,y label
plt.ylabel(r'Pressure [MPa]', fontsize=16, fontweight='bold')
plt.xlabel(r'HFC-32 in IL [mol%]', fontsize=16, fontweight='bold')
plt.legend()
plt.show()